# Change tensorflow version from 2.6.4 to 2.9.1

I noticed that the default version of tensorflow (2.6.4) has a problem of printing unecessary clean up messages. for this reason I upgraded to tensorflow 2.9.1

In [ ]:
!pip uninstall tensorflow --yes
!pip install tensorflow_decision_forests
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2 --yes
!pip install kerassurgeon 

# Library
here are the required packages/modules we will use to perform the tasks

In [ ]:
import os
import cv2
import gc
import numpy as np
import pandas as pd
import glob
import pathlib
import tensorflow as tf
import tensorflow_addons as tfa
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator
#from keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras import datasets, layers, models,Input,Model 
import tensorflow_datasets as tfds
from keras.models import Sequential
from keras.layers import  Bidirectional, Conv2D, BatchNormalization, MaxPooling2D, Flatten, LSTM, Dense, Lambda, Dropout,Reshape
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from tensorflow.keras.metrics import Accuracy, Recall,Precision
from sklearn.tree import DecisionTreeClassifier as Decisiontree
from sklearn.svm import SVC as Supportvectorclassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time
from functools import reduce
from kerassurgeon.operations import delete_layer, insert_layer
from keras.utils import to_categorical
import pickle

Just checking if tensorflow was sucessfully upgraded.

In [ ]:
print(tf.__version__)

In [ ]:
# some parameter settings

batch_size=32
image_height=299
image_width=299
n=1000
epochs=100

In [ ]:
# function to get labs 
def fetch_labels(filepath):
    return str(filepath).split('/')[-2]


 # Dataset 3: Patch Camelyon

In [ ]:
ds,info=tfds.load("PatchCamelyon", with_info=True,as_supervised = True)  

In [ ]:
print(info)

In [ ]:
train_data = ds['train'].batch(batch_size=32)
valid_data = ds['validation'].batch(batch_size=32)
test_data = ds['test'].batch(batch_size=32)
train_ds1=train_data.take(n)

In [ ]:
lebb=['No metastatic tissue','metastatic tissue']

def to_lab(x):
    return lebb[round(x)]

def to_lab1(batched):
    labs=np.array([])

    for batch in batched:
        images,labels=batch
        lb=np.array(list(map(to_lab,labels.numpy())))
        labs= np.append(labs,lb)
    return labs

In [ ]:
trainlabs=to_lab1(train_data)
testlabs=to_lab1(test_data)
validlabs=to_lab1(valid_data)

In [ ]:
fulld=pd.concat([pd.DataFrame({'Label': trainlabs}),
           pd.DataFrame({'Label': testlabs}),
           pd.DataFrame({'Label': validlabs})])

figure,ax=plt.subplots(nrows=2,ncols=3,figsize=(12,12))

sns.countplot(x = 'Label',data = pd.concat([pd.DataFrame({'Label': trainlabs}),
           pd.DataFrame({'Label': testlabs}),
           pd.DataFrame({'Label': validlabs})]),ax=ax[0,0])

sns.countplot(x = 'Label',data = pd.DataFrame({'Label': trainlabs}),ax=ax[0,1])
sns.countplot(x='Label',data=pd.DataFrame({'Label': testlabs}),ax=ax[0,2])
sns.countplot(x='Label',data=pd.DataFrame({'Label': validlabs}),ax=ax[1,0])
sns.barplot(x = 'Label',y = 'count',
            data = pd.DataFrame({'Label':['Training',
                                          'Testing',
                                          'Validation'],
                                 'count': [len(train_data),
                                           len(test_data),
                                           len(valid_data)]}),ax=ax[1,1])
ax[0,0].set_title('Total Images')
ax[0,1].set_title('Training images')
ax[0,2].set_title('Testing images')
ax[1,0].set_title('validation images')
ax[1,1].set_title('Batch counts')

for p, label in zip(ax[0,0].patches, fulld['Label'].value_counts().index):
    ax[0,0].annotate(round(p.get_height()), (p.get_x()+0.25, p.get_height()+5))
                    
for p, label in zip(ax[0,1].patches, pd.value_counts(trainlabs).index):
    ax[0,1].annotate(p.get_height(), (p.get_x()+0.25, p.get_height()+20))
    
for p, label in zip(ax[0,2].patches, pd.value_counts(trainlabs).index):
    ax[0,2].annotate(p.get_height(), (p.get_x()+0.25, p.get_height()+20))
    
for p, label in zip(ax[1,0].patches, pd.value_counts(testlabs).index):
    ax[1,0].annotate(round(p.get_height()), (p.get_x()+0.25, p.get_height()+20))
    
for p, label in zip(ax[1,1].patches, pd.DataFrame({'Label':['Training',
                                          'Testing',
                                          'Validation'],
                                 'count': [len(train_data),
                                           len(test_data),
                                           len(valid_data)]})['Label'].value_counts().index):
    ax[1,1].annotate(round(p.get_height()), (p.get_x()+0.25, p.get_height()+20))
    
                    
ax[0,0].set_xlabel("")
ax[0,1].set_xlabel("")
ax[0,2].set_xlabel("")
ax[1,0].set_xlabel("")
ax[1,1].set_xlabel("")
plt.savefig('Patch camelyon data1.png')

In [ ]:
plt.figure(figsize=(13, 13))
for data in train_ds1.take(1):
    images, labels = data
    for i in range(16):
        ls=labels[i].numpy()
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"),cmap='Greys_r')
        plt.title(lebb[ls])
        plt.axis("off")
plt.savefig('Patch camelyon data2.png')
        

In [ ]:
def to_categorical_crossentr(image, label):
    return tf.image.rgb_to_grayscale(image),tf.one_hot(label, 2,dtype=tf.int64)

train_data = ds['train'].map(to_categorical_crossentr).batch(batch_size=32)
valid_data = ds['validation'].map(to_categorical_crossentr).batch(batch_size=32)
test_data = ds['test'].map(to_categorical_crossentr).batch(batch_size=32)

In [ ]:
# performance configuration
AUTOTUNE = tf.data.AUTOTUNE
train_ds1 = train_data.cache().prefetch(buffer_size=AUTOTUNE)
test_ds1 = test_data.cache().prefetch(buffer_size=AUTOTUNE)
test_ds1 = test_data.cache().prefetch(buffer_size=AUTOTUNE) 
train_ds1=train_ds1.take(n)

In [ ]:
model10=Sequential(name="Full_Model10")
# Block 1
model10.add(Input(shape=(96,96,1),name="input"))
model10.add(Conv2D(64, (3, 3), padding='same', activation='relu',name="block1_conv_1"))
model10.add(BatchNormalization(name="block1_batch_normalization1"))
model10.add(Conv2D(64, (3, 3), padding='same', activation='relu',name="block1_conv_2"))
model10.add(BatchNormalization(name="block1_batch_normalization2"))
model10.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2),name="block1_maxpool"))

# Block 2

model10.add(Conv2D(128, (3, 3), padding='same', activation='relu',name="block2_conv_1"))
model10.add(BatchNormalization(name="block2_batch_normalization1"))
model10.add(Conv2D(128, (3, 3), padding='same', activation='relu',name="block2_conv_2"))
model10.add(BatchNormalization(name="block2_batch_normalization2"))
model10.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2),name="block2_maxpool"))
          
# Block 3
model10.add(Conv2D(256, (3, 3), padding='same', activation='relu',name="block3_conv_1"))
model10.add(BatchNormalization(name="block3_batch_normalization1"))
model10.add(Conv2D(256, (3, 3), padding='same', activation='relu',name="block3_conv_2"))
model10.add(BatchNormalization(name="block3_batch_normalization2"))
model10.add(Conv2D(256, (3, 3), padding='same', activation='relu',name="block3_conv_3"))
model10.add(BatchNormalization(name="block3_batch_normalization3"))
model10.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),name="block3_maxpool"))


# Block 4
model10.add(Conv2D(512, (3, 3), padding='same', activation='relu',name="block4_conv_1"))
model10.add(BatchNormalization(name="block4_batch_normalization1"))
model10.add(Conv2D(512, (3, 3), padding='same', activation='relu',name="block4_conv_2"))
model10.add(BatchNormalization(name="block4_batch_normalization2"))
model10.add(Conv2D(512, (3, 3), padding='same', activation='relu',name="block4_conv_3"))
model10.add(BatchNormalization(name="block4_batch_normalization3"))
model10.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2),name="block4_maxpool"))

# fifth convolution layer

model10.add(Conv2D(512, (3, 3), padding='same', activation='relu',name="block5_conv_1"))
model10.add(BatchNormalization(name="block5_batch_normalization1"))
model10.add(Conv2D(512, (3, 3), padding='same', activation='relu',name="block5_conv_2"))
model10.add(BatchNormalization(name="block5_batch_normalization2"))
model10.add(Conv2D(512, (3, 3), padding='same', activation='relu',name="block5_conv_3"))
model10.add(BatchNormalization(name="block5_batch_normalization3"))
model10.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2),name="block5_maxpool"))
#  flatten
model10.add(Flatten(name="flatten_layer"))

# Dense connected layers
model10.add(Dense(units=64,activation="relu"))
model10.add(Dense(units=64,activation="relu"))
model10.add(Dense(units=2, activation="softmax"))

In [ ]:
model10.compile(
  optimizer='adam',
  loss="categorical_crossentropy",
  metrics=["accuracy",
           tf.keras.metrics.Recall(name="Sensitivity",class_id=0),
           tf.keras.metrics.Recall(name="Specificity",class_id=1),
           tf.keras.metrics.Precision(name="Precision",class_id=0),
          tfa.metrics.F1Score(num_classes=2, average="micro")])

In [ ]:
hist=model10.fit(
 train_ds1,
  validation_data=test_ds1,
 epochs=epochs)

In [ ]:
#Accuracy
figure,ax=plt.subplots(nrows=1,ncols=2,figsize=(15,6))
ax[0].plot(hist.history['accuracy'])
ax[0].plot(hist.history['val_accuracy'])
ax[0].set_title('model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].legend(['train', 'test'], loc='upper left')

# loss
ax[1].plot(hist.history['loss'])
ax[1].plot(hist.history['val_loss'])
ax[1].set_title('model loss')
ax[1].set_ylabel('loss')
ax[1].set_xlabel('epoch')
ax[1].legend(['train', 'test'], loc='upper left')
plt.savefig('Patch camelyon data3.png')

In [ ]:
#m1=model.evaluate(test_ds)
st = time.time()
m1=model10.evaluate(test_ds1)[1:6]
et = time.time()
elapsed_time=round((et - st)/len(testlabs),4)
print('Execution time:', elapsed_time, 'seconds')
m1.append(elapsed_time)
mod4=pd.DataFrame({"Measure":['Accuracy','Sensitivity','Specificty','Precision',"F1-score","Excecution time"],
    "Patch camelyon data":[np.round(float(i), 4) for i in m1]})
mod4

In [ ]:
model10.save("patchcamelyon")

with open("patchcamelyon.pkl","wb") as file:
    pickle.dump(hist,file) 